In [2]:
pip install flask requests


Note: you may need to restart the kernel to use updated packages.


In [5]:
import threading
import time
from flask import Flask, Response, request
import requests

# --- 1. SETUP FLASK MOCK SERVER ---

app = Flask(__name__)

@app.route('/mcp', methods=['POST'])
def mcp_handler():
    data = request.json
    print("\n📥 Received request:", data)

    def stream():
        for i in range(3):
            time.sleep(1)
            yield f"data: {{\"message\": \"Update {i+1}\"}}\n\n"
            yield ": keep-alive\n\n"  # Force immediate flush

    return Response(stream(), mimetype='text/event-stream')

@app.route('/')
def home():
    return "🏠 MCP Mock Server Running!"

def start_server():
    app.run(port=5000)

# --- 2. START SERVER IN BACKGROUND THREAD ---
server_thread = threading.Thread(target=start_server)
server_thread.daemon = True
server_thread.start()

# --- 3. WAIT FOR SERVER TO BE READY ---
time.sleep(3)  # Give Flask server a few seconds to boot

print("\n✅ Server started at http://127.0.0.1:5000")

# --- 4. CLIENT CODE: CONNECT TO SERVER ---

try:
    SERVER_URL = "http://127.0.0.1:5000"

    response = requests.post(f"{SERVER_URL}/mcp", stream=True, json={
        "tool": "example_tool",
        "input": {"message": "Hello via SSE!"}
    })

    print("\n🚀 Connecting to SSE Server...\n")
    event_count = 0
    for line in response.iter_lines():
        if line:
            decoded_line = line.decode('utf-8')
            print(decoded_line)
            event_count += 1
        if event_count >= 3:
            break  # Stop after 3 messages

    print("\n🎯 Successfully received all SSE events!")

except Exception as e:
    print("❌ Connection failed:", e)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit



✅ Server started at http://127.0.0.1:5000

📥 Received request: {'tool': 'example_tool', 'input': {'message': 'Hello via SSE!'}}


127.0.0.1 - - [28/Apr/2025 17:31:39] "POST /mcp HTTP/1.1" 200 -



🚀 Connecting to SSE Server...

data: {"message": "Update 1"}
: keep-alive
data: {"message": "Update 2"}

🎯 Successfully received all SSE events!
